# Meta programming

> Objects are created or modified by other objects

In [16]:
def test(name: str):
    print(name)

In [17]:
test.__annotations__

{'name': str}

In [23]:
def test(name: str, test: int):
    print(name, test)

In [19]:
test.__annotations__

{'name': str, 'test': int}

In [ ]:
from typing import List

In [21]:
def test(name: str, test: List[int]):
    print(name, test)

In [22]:
test.__annotations__

{'name': str, 'test': typing.List[int]}

In [24]:
def test(name: str, test: List[int]) -> int:
    print(name, test)
    return 0

In [25]:
test.__annotations__

{'name': str, 'return': int, 'test': typing.List[int]}

## Exercise

Write a meta function that takes a function documentation and adds input and output as found in annotations

# Meta programming with classes

In [51]:
from beeprint import pp # pretty print

In [10]:
class SimpleClass: pass

In [12]:
class SimpleClassExplicit(object):
    pass

In [96]:
pp(SimpleClass)

class(SimpleClass)


In [15]:
SimpleClass.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'SimpleClass' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'SimpleClass' objects>})

In [16]:
# metaclassing
SimpleClassMeta = type('SimpleClassMeta', (), {})

In [18]:
SimpleClassMeta.__dict__

mappingproxy({'__dict__': <attribute '__dict__' of 'SimpleClassMeta' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'SimpleClassMeta' objects>})

In [20]:
# adding an attribute after class creation
SimpleClassMeta.x = 42

In [23]:
pp(SimpleClassMeta)

class(SimpleClassMeta):
  x: 42


In [29]:
class SimpleClassExplicit(object):
    x = 42

In [30]:
pp(SimpleClassExplicit)

class(SimpleClassExplicit):
  x: 42


In [27]:
# adding an attribute while creating the class
SimpleClassMeta2 = type('SimpleClassMeta2', (), {'x': 42})

In [28]:
pp(SimpleClassMeta2)

class(SimpleClassMeta2):
  x: 42


In [62]:
SimpleClassMeta2 = type('SimpleClassMeta2', (), dict(x=42, y=True))

In [63]:
pp(SimpleClassMeta2)

class(SimpleClassMeta2):
  x: 42,
  y: True


In [64]:
# What about the object instance?
instance = SimpleClassMeta2()

In [65]:
pp(instance)

instance(SimpleClassMeta2):
  x: 42,
  y: True


In [66]:
SimpleClassMeta2 = type('SimpleClassMeta2', (list,), dict(x=42, y=True))

In [68]:
pp(SimpleClassMeta2)

class(SimpleClassMeta2):
  append: <method 'append' of 'list' objects>,
  clear: <method 'clear' of 'list' objects>,
  copy: <method 'copy' of 'list' objects>,
  count: <method 'count' of 'list' objects>,
  extend: <method 'extend' of 'list' objects>,
  index: <method 'index' of 'list' objects>,
  insert: <method 'insert' of 'list' objects>,
  pop: <method 'pop' of 'list' objects>,
  remove: <method 'remove' of 'list' objects>,
  reverse: <method 'reverse' of 'list' objects>,
  sort: <method 'sort' of 'list' objects>,
  x: 42,
  y: True


what happened?

In [69]:
type?

In [70]:
instance = SimpleClassMeta2()

In [71]:
pp(instance)

[]


In [40]:
class SimpleClassExplicit(object):
    x = 42

In [86]:
SimpleClassMeta = type('SimpleClassMeta', (SimpleClassExplicit,), dict(y=True))

In [87]:
pp(SimpleClassMeta)

class(SimpleClassMeta):
  x: 42,
  y: True


In [88]:
SimpleClassMeta.__dict__

mappingproxy({'__doc__': None, '__module__': '__main__', 'y': True})

## Exercise

Find which internal attribute of the SimpleClassMeta listes the parent classes.


Can you find the opposite too?

In [73]:
# setting an attribute only to its instance
def myinit(self):
    self.x = 42

In [75]:
SimpleClassMeta = type('SimpleClassMeta', (), dict(__init__=myinit))

In [76]:
pp(SimpleClassMeta)

class(SimpleClassMeta)


In [77]:
pp(SimpleClassMeta())

instance(SimpleClassMeta):
  x: 42


## Meta Attributes handling

> getattr, setattr, hasattr

In [78]:
setattr(SimpleClassMeta, 'y', True)

In [79]:
hasattr(SimpleClassMeta, 'y')

True

In [81]:
getattr(SimpleClassMeta, 'y')

True

In [80]:
hasattr(SimpleClassMeta, 'w')

False

In [82]:
getattr(SimpleClassMeta, 'w')

AttributeError: type object 'SimpleClassMeta' has no attribute 'w'

In [85]:
getattr(SimpleClassMeta, 'w', None) # default

# Disassembling

<small>
src: https://www.getdrip.com/deliveries/cvxdqhqopuxpws94ayrx?__s=ug11p5y75reem5tszuoo
</small>


In [1]:
def greet(name):
    return 'Hello, ' + name + '!'

In [2]:
greet('Dan')

'Hello, Dan!'

In [3]:
import dis

In [4]:
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 BINARY_ADD
              6 LOAD_CONST               2 ('!')
              8 BINARY_ADD
             10 RETURN_VALUE


In [7]:
def explode(name):
    
    for char in name:
        print(char)
        
    return list(name)

In [8]:
explode("Dan")

D
a
n


['D', 'a', 'n']

In [9]:
dis.dis(explode)

  3           0 SETUP_LOOP              20 (to 22)
              2 LOAD_FAST                0 (name)
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_FAST               1 (char)

  4          10 LOAD_GLOBAL              0 (print)
             12 LOAD_FAST                1 (char)
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK

  6     >>   22 LOAD_GLOBAL              1 (list)
             24 LOAD_FAST                0 (name)
             26 CALL_FUNCTION            1
             28 RETURN_VALUE


In [5]:
def apply_some_function_to_a_number(func):
    def wrapper(x):            
        y = random.randint(1, 10)
        print("x: %s ^ y: %s" % (x, y))
        return func(x, y)
    return wrapper

In [6]:
dis.dis(apply_some_function_to_a_number)

  2           0 LOAD_CLOSURE             0 (func)
              2 BUILD_TUPLE              1
              4 LOAD_CONST               1 (<code object wrapper at 0x7fd378a5b8a0, file "<ipython-input-5-91e3a6b23e2d>", line 2>)
              6 LOAD_CONST               2 ('apply_some_function_to_a_number.<locals>.wrapper')
              8 MAKE_FUNCTION            8
             10 STORE_FAST               1 (wrapper)

  6          12 LOAD_FAST                1 (wrapper)
             14 RETURN_VALUE


In [11]:
for a in ['a', "test", 1, False]:
    a + 'string'

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [13]:
# last traceback
dis.dis()

  1           0 SETUP_LOOP              20 (to 22)
              2 LOAD_CONST               6 (('a', 'test', 1, False))
              4 GET_ITER
        >>    6 FOR_ITER                12 (to 20)
              8 STORE_NAME               0 (a)

  2          10 LOAD_NAME                0 (a)
             12 LOAD_CONST               4 ('string')
    -->      14 BINARY_ADD
             16 POP_TOP
             18 JUMP_ABSOLUTE            6
        >>   20 POP_BLOCK
        >>   22 LOAD_CONST               5 (None)
             24 RETURN_VALUE


# Debugger

<small>
src: https://github.com/spiside/pdb-tutorial
</small>

# End of Chapter